# Group Details

## Group Name: Group 08

### Student 1: Jasper Linders

### Student 2: Alexander Liu

### Student 3: Sjoerd van Straten

# Loading Data and Preliminaries

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [5]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time.
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_array('data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-5.85725792 -5.394571           nan         nan]
 [-6.03781257 -5.72445953         nan         nan]
 [-0.90623054 -6.93416278         nan         nan]
 [ 2.83149339 -7.50100819         nan         nan]
 [-2.85586881  1.77667501         nan         nan]
 [ 4.04424526  4.00563603         nan         nan]
 [-5.24887713 -4.83081005         nan         nan]
 [-5.81391023 -5.1109838          nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


In [6]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""
example = load_array('data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


# Data Handling and Preprocessing

First, let's import the data and make variables for the train and test data.

In [9]:
import os
from torch.utils.data import Dataset

class ImportData(Dataset):
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.file_list = sorted(os.listdir(folder_path))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        file_name = self.file_list[index]
        file_path = os.path.join(self.folder_path, file_name)
        data, label = load_array(file_path, task='task 1')
        return data, label

# Create an instance of the custom dataset class with the folder path
train_import = ImportData('data/task 1/train/')
test_import = ImportData('data/task 1/test/')

X_train_import = []
y_train_import = []
X_test_import = []
y_test_import = []

# Iterate through the train_dataset to extract data and labels
for data, label in train_import:
    X_train_import.append(data)
    y_train_import.append(label)

for data, label in test_import:
    X_test_import.append(data)
    y_test_import.append(label)

max_length = 9

# Pad the data samples with zeros to have the same shape
X_train_padded = []
for data in X_train_import:
    pad_width = max_length - data.shape[0]
    padded_data = np.pad(data, ((0, pad_width), (0, 0)), mode='constant')
    X_train_padded.append(padded_data)

y_train_padded = []
for label in y_train_import:
    pad_width = max_length - label.shape[0]
    padded_label = np.pad(label, ((0, pad_width), (0, 0)), mode='constant')
    y_train_padded.append(padded_label)

X_test_padded = []
for data in X_test_import:
    pad_width = max_length - data.shape[0]
    padded_data = np.pad(data, ((0, pad_width), (0, 0)), mode='constant')
    X_test_padded.append(padded_data)

y_test_padded = []
for label in y_test_import:
    pad_width = max_length - label.shape[0]
    padded_label = np.pad(label, ((0, pad_width), (0, 0)), mode='constant')
    y_test_padded.append(padded_label)

# Convert the padded data and labels to tensors
X_train = torch.tensor(np.array(X_train_padded))
y_train = torch.tensor(np.array(y_train_padded))
X_test = torch.tensor(np.array(X_test_padded))
y_test = torch.tensor(np.array(y_test_padded))

# Print the shape of X_train and the first label in y_train
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  torch.Size([900, 9, 5])
y_train shape:  torch.Size([900, 9, 2])
X_test shape:  torch.Size([100, 9, 5])
y_test shape:  torch.Size([100, 9, 2])


In [30]:
import torch
from torch.utils.data import Dataset, DataLoader

# Function to process a single data point
def create_graph(data_point):

    #Create node features
    node_features = data_point  # Exclude the mass column

    # Compute edge indices
    edge_indices = []
    for i in range(9):
        for j in range(9):
            if i != j:
                edge_indices.append([i, j])
    edge_indices = torch.tensor(edge_indices).t().contiguous()

    # Compute edge features
    edge_features = []
    for i in range(9):
        for j in range(9):
            if i != j:
                pos = node_features[j] - node_features[i]

                # Calculate Euclidean distance
                # pos = torch.tensor(pos)
                relative_position = torch.norm(pos)
                # print("Relative position: ", relative_position)

                relative_mass = node_features[j, 0] / node_features[i, 0]
                if relative_mass == np.inf or relative_mass == 0:
                    relative_position = 0
                    relative_mass = 0
                # relative_mass = torch.tensor(relative_mass)
                # print("Relative mass: ", relative_mass)
                edge_features.append([relative_position, relative_mass])
                # edge_features.append(torch.cat((relative_position, torch.tensor([relative_mass], dtype=torch.float32))))
    edge_features = torch.FloatTensor(edge_features)

    data = Data(x=node_features, edge_index=edge_indices, edge_attr=edge_features)

    return data

In [31]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# Load train data into a list of Data objects
train_graphs_lst = []
for i in range(len(X_train)):
    train_graphs_lst.append(create_graph(X_train[i]))

# Create train DataLoader
train_dataloader = DataLoader(train_graphs_lst, batch_size=1, shuffle=True)

# Load test data into a list of Data objects
test_graphs_lst = []
for i in range(len(X_test)):
    test_graphs_lst.append(create_graph(X_test[i]))

# Create test DataLoader
test_dataloader = DataLoader(test_graphs_lst, batch_size=1, shuffle=False)

In [26]:
train_dataloader.dataset[0]

Data(x=[9, 5], edge_index=[2, 72], edge_attr=[72, 2])

In [29]:
# import networkx as nx
# import matplotlib.pyplot as plt

# value, label = train_dataloader.dataset[2]
# node_features, edge_indices = create_graph(value)

# # Create an empty graph
# graph = nx.Graph()

# # Add nodes with node numbers as labels
# for i in range(node_features.shape[0]):
#     graph.add_node(i, label=i)

# # Add edges
# for edge in edge_indices.t().tolist():
#     graph.add_edge(*edge)

# # Plot the graph
# pos = nx.spring_layout(graph, seed=42)  # Layout for node positioning
# nx.draw_networkx(graph, pos=pos, with_labels=True)

# # Add node numbers as text labels
# labels = nx.get_node_attributes(graph, "label")
# nx.draw_networkx_labels(graph, pos=pos, labels=labels)

# # Add node features as text labels
# for i, feature in enumerate(node_features):
#     x, y = feature[:2]
#     plt.text(pos[i][0], pos[i][1], f"({x:.2f}, {y:.2f})", ha="center", va="bottom")

# # Display the plot
# plt.axis("off")
# plt.show()

# Model Implementation

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Model Training

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN(in_channels=5, hidden_channels=16, out_channels=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

for epoch in range(10):
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index)  # Pass the node features to the model
        loss = criterion(output, batch.y)
        loss.backward()
        optimizer.step()

    model.eval()
    # Evaluation code goes here

RuntimeError: expected scalar type Double but found Float

# Evaluation